In [2]:
# Import libraries
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

## Brand Data
Read in brands.json and put in to pandas dataframes

In [327]:
# read in data
brands = pd.read_json("../Data/brands.json",lines=True)
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [328]:
# clean up columns
clean_ids = pd.json_normalize(brands['_id'])
clean_cpgs = pd.json_normalize(brands['cpg'])
brands.drop(columns = ['_id','cpg'])
brands['_id'] = clean_ids 
brands['$ref'] = clean_cpgs['$ref']
brands['cpg'] = clean_cpgs['$id.$oid']
brands

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,$ref
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,601ac114be37ce2ead437550,test brand @1612366101024,0.0,NaN,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Starbucks,0.0,STARBUCKS,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs
...,...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,5f77274dbe37ce6b592e90bf,test brand @1601644365844,NaN,NaN,Cogs
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,53e10d6368abd3c7065097cc,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,Cogs
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,Cogs
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,5332f5f6e4b03c9a25efd0b4,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,Cogs


In [329]:
# Create tables for relational database

# barcodes and associated brands
barcodes = brands[['_id','barcode']].drop_duplicates()
barcodes['brandNumber'] = df.groupby(['_id']).cumcount()+1
barcodes['brandNumber'] = df.groupby(['_id']).cumcount()+1;
barcodes.pivot_table(index=["barcode"], 
                     columns='brandNumber', 
                     values='grade')
# cpgs and reference
cpgs = brands[['$ref','cpg']]
cpgs = cpgs.drop_duplicates()
cpgs.sort_values(by=['cpg'], inplace=True)
cpgs.reset_index(inplace=True)
cpgs['cpgId'] = cpgs.index + 1
cpgs.drop(columns=['index'], inplace=True)

# category and category code lookup for brands
categories = barcodes[['category', 'categoryCode']]
categories = categories.drop_duplicates()
categories.sort_values(by=['category'], inplace=True)
categories.reset_index(inplace=True)
categories['categoryId'] = categories.index + 1
categories.drop(columns=['index'], inplace=True)
categories

# merge on cpg and category codes to brands table
brands = brands.merge(cpgs, on=['$ref', 'cpg'], how='left')
brands = brands.merge(categories, on=['category', 'categoryCode'], how='left')
brands.drop(columns = ['category', 'categoryCode', '$ref', 'cpg'], inplace=True)
brands = brands.drop_duplicates()
brands

,_id,barcode,cpgId
0,601ac115be37ce2ead437551,511111019862,202
1,601c5460be37ce2ead43755f,511111519928,7
2,601ac142be37ce2ead43755d,511111819905,203
3,601ac142be37ce2ead43755a,511111519874,203
4,601ac142be37ce2ead43755e,511111319917,17
...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,113
1163,5dc1fca91dda2c0ad7da64ae,511111706328,24
1164,5f494c6e04db711dd8fe87e7,511111416173,17
1165,5a021611e4b00efe02b02a57,511111400608,6


## User Data
Read in users.json and put in to pandas dataframes

In [330]:
# read in data
users = pd.read_json("../Data/users.json",lines=True)
users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
...,...,...,...,...,...,...,...
490,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
491,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
492,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
493,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN


In [331]:
# clean up columns
clean_ids = pd.json_normalize(users['_id'])
clean_date1 = pd.json_normalize(users['createdDate'])
users['lastLogin'] = users['lastLogin'].fillna({i: {} for i in users.index})
clean_date2 = pd.json_normalize(users['lastLogin'])
users.drop(columns = ['_id','createdDate','lastLogin'])
users['_id'] = clean_ids 
users['createdDate'] = clean_date1['$date']
users['lastLogin'] = clean_date2['$date']
users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,NaN,NaN


## Receipts Data
Read in receipts.json and put in to pandas dataframes

In [332]:
# read in data
receipts = pd.read_json("../Data/receipts.json",lines=True)
receipts

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,{'$oid': '603cc0630a720fde100003e6'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614594147000},{'$date': 1614594147000},NaN,{'$date': 1614594148000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},NaN,{'$date': 1614613361873},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},NaN,{'$date': 1614607657664},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [333]:
# clean up columns
clean_ids = pd.json_normalize(receipts['_id'])
clean_date1 = pd.json_normalize(receipts['createDate'])
clean_date2 = pd.json_normalize(receipts['dateScanned'])
receipts['finishedDate'] = receipts['finishedDate'].fillna({i: {} for i in receipts.index})
clean_date3 = pd.json_normalize(receipts['finishedDate'])
clean_date4 = pd.json_normalize(receipts['modifyDate'])
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].fillna({i: {} for i in receipts.index})
clean_date5 = pd.json_normalize(receipts['pointsAwardedDate'])
receipts['purchaseDate'] = receipts['purchaseDate'].fillna({i: {} for i in receipts.index})
clean_date6 = pd.json_normalize(receipts['purchaseDate'])
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].replace(np.nan, '', regex=True)
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].fillna({i: {} for i in receipts.index})
receipts.drop(columns = ['_id','createDate','dateScanned','finishedDate','modifyDate','pointsAwardedDate','purchaseDate','rewardsReceiptItemList'])
receipts['_id'] = clean_ids 
receipts['createDate'] = clean_date1['$date']
receipts['dateScanned'] = clean_date2['$date']
receipts['finishedDate'] = clean_date3['$date']
receipts['modifyDate'] = clean_date4['$date']
receipts['pointsAwardedDate'] = clean_date5['$date']
receipts['purchaseDate'] = clean_date6['$date']
receipts

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5.0,1.609632e+12,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,1614594147000,1614594147000,NaN,1614594148000,NaN,25.0,1.597622e+12,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,1614613361873,1614613361873,NaN,1614613361873,NaN,NaN,NaN,NaN,,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,1614607657664,1614607657664,NaN,1614607657664,NaN,NaN,NaN,NaN,,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,1614604048000,1614604048000,NaN,1614604049000,NaN,25.0,1.597622e+12,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [346]:
# clean receipt item list
item_list = receipts[['_id','rewardsReceiptItemList']]
item_list_clean = pd.DataFrame()
y = 0
for i, x in zip(item_list['_id'],item_list['rewardsReceiptItemList']):
    print(y)
    for item in x:
        if x == "":
            item_list_clean = item_list_clean.append({'_id':i,'barcode':'','description':'','finalPrice':'','needsFetchReview':'','partnerItemId':'','preventTargetGapPoints':'','quantityPurchased':'','userFlaggedBarcode':'','userFlaggedNewItem':'','userFlaggedPrice':'','userFlaggedQuantity':''}, ignore_index=True)
        else:
            tmp = pd.json_normalize(x)
            tmp['_id'] = i
            item_list_clean = item_list_clean.append(tmp)
    y += 1
item_list_clean

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
0,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
1,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
0,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


In [352]:
receipt_items = item_list_clean
receipts.drop(columns=['rewardsReceiptItemList'], inplace=True)

### Save tables to CSV

In [357]:
tables = [(brands,'brands'),(categories,'categories'),(users,'users'),(cpgs,'cpgs'),(receipts,'receipts'),(receipt_items, 'receipt_items')]
for table in tables:
    table[0].to_csv("../CSV Tables/{}.csv".format(table[1]))